## Explicación de NNLLLoss y CrossEntropyLoss


### [CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

CrossEntropyLoss es ampliamente utilizado en tareas de clasificación donde las salidas son probabilidades que suman uno. Mide la disimilitud entre la distribución de etiquetas verdaderas y las predicciones, penalizando las probabilidades en función de cuán lejos están de la etiqueta real.

**Ecuación:**
$$ \text{CrossEntropyLoss} = -\sum_{c=1}^M y_{o,c} \log(p_{o,c}) $$

Donde:
- $ M $ es el número de clases.
- $ y_{o,c} $ es un indicador binario (0 o 1) si la etiqueta de clase $ c $ es la clasificación correcta para la observación $ o $.
- $ p_{o,c} $ es la probabilidad predicha de que la observación $ o $ pertenezca a la clase $ c $.

**Ejemplo:**
Supongamos que tenemos un problema de clasificación de 3 clases y la etiqueta verdadera para una instancia es la clase 2, representada como codificada en caliente `[0, 1, 0]`. Si las probabilidades predichas para las clases son `[0.2, 0.7, 0.1]`, el CrossEntropyLoss se calcula como:

$$ -\left(0 \times \log(0.2) + 1 \times \log(0.7) + 0 \times \log(0.1)\right) = -\log(0.7) \approx 0.357 $$

**Nota**: PyTorch lo que calcula realmente es $ \text{CrossEntropyLoss} = -\sum_{c=1}^M y_{o,c} \log\left(\frac{e^{l_{o,c}}}{\sum_{j=1}^M e^{l_{o,j}}}\right)$. Donde $ l_{o,c} $ son los logits (puntuaciones sin procesar) para la clase $ c $ de la observación $o$. Es decir que, PyTorch aplica internamente la función softmax a los logits antes de calcular el logaritmo de las probabilidades, lo cual es parte integral del cálculo de CrossEntropyLoss.


### [NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss) (Negative Log Likelihood Loss)

NLLLoss se utiliza cuando las salidas del modelo son logaritmos de probabilidades. Esta función de pérdida es más simple ya que utiliza directamente los logaritmos de probabilidades proporcionados por el modelo, enfocándose específicamente en el logaritmo de la probabilidad de la clase verdadera.

**Ecuación:**
$$ \text{NLLLoss} = -\text{LP}_{\text{claseVerdad}} $$

Donde:
- $ \text{LP}_{\text{claseVerdad}} $ es el logaritmo de la probabilidad de la clase verdadera según lo proporciona la salida del modelo.

**Ejemplo:**
Continuando con la configuración anterior, si el modelo produce logaritmos de probabilidades para las clases como `[-1.61, -0.3567, -2.30]` (correspondiendo a las probabilidades `[0.2, 0.7, 0.1]`), y la clase verdadera es la segunda, el NLLLoss es:

$$ \text{NLLLoss} = -(-0.3567) = 0.3567 $$

### Resumen

- **CrossEntropyLoss** se utiliza cuando las salidas del modelo son puntuaciones brutas (logits) o probabilidades. Aplica una softmax para convertir logits en probabilidades, seguido de calcular el logaritmo de estas probabilidades, y finalmente calcula la pérdida de logaritmo negativo.
- **NLLLoss** se utiliza cuando las salidas del modelo ya están en forma de logaritmos de probabilidades. Calcula directamente el negativo del logaritmo de la probabilidad de la clase correcta.

Estas funciones de pérdida guían al modelo durante el entrenamiento penalizando las predicciones basadas en cuánto se desvían de las etiquetas reales, ayudando así en el aprendizaje de las correspondencias correctas de entradas a salidas.

| Nombre de la Pérdida                  | Cuándo se usa                         | Cómo son los Inputs                          |
|---------------------------------------|---------------------------------------|----------------------------------------------|
| CrossEntropyLoss                      | Clasificación multiclase              | Logits (puntuaciones sin procesar)           |
| NLLLoss                               | Clasificación multiclase con probabilidades logarítmicas | Logaritmos de probabilidades (después de LogSoftmax) |
| BCELoss          | Clasificación binaria                 | Probabilidades (después de una función sigmoide) |
| BCEWithLogitsLoss      | Clasificación binaria                 | Logits (puntuaciones sin procesar)           |

## Explicación del Optimizador Adam

#### Introducción a SGD y Momentum

Para entender Adam, primero es útil comprender el optimizador de Gradiente Descendente Estocástico (SGD) y el concepto de momentum. SGD es un método para optimizar una función objetivo, típicamente una función de pérdida en aprendizaje automático, que se actualiza en base a los gradientes de la función con respecto a los parámetros del modelo. La actualización básica de SGD se puede expresar como:

$$\theta_{t+1} = \theta_t - \rho \nabla_\theta J(\theta_t)$$

Donde:
- $\theta$ son los parámetros del modelo.
- $\rho$ es la tasa de aprendizaje.
- $J$ es la función de pérdida.
- $\nabla_\theta J(\theta_t)$ es el gradiente de la función de pérdida respecto a $\theta$ en el tiempo $t$.

**Momentum** es una técnica para acelerar SGD en la dirección correcta y suavizar las actualizaciones. Se añade un término de "velocidad" $v_t$ que incorpora gradientes pasados:

$$v_{t+1} = \mu v_t + \rho \nabla_\theta J(\theta_t)$$
$$\theta_{t+1} = \theta_t - v_{t+1}$$

Donde $\mu$ (típicamente alrededor de 0.9) es el factor de momentum que determina la contribución de gradientes pasados.

#### Adam: Adaptive Moment Estimation

Adam, que significa "Adaptive Moment Estimation", combina las ideas de momentum y escalado adaptativo de los gradientes por segundo momento. Adam mantiene estimaciones del primer momento (el promedio de los gradientes, similar a momentum) y del segundo momento (el promedio de los cuadrados de los gradientes) de los gradientes.

Las ecuaciones de Adam son:

1. **Calcular los gradientes**:
   $$g_t = \nabla_\theta J(\theta_t)$$

2. **Actualizar los momentos estimados**:
   $$m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t$$
   $$v_t = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2$$

   Donde:
   - $m_t$ es el primer momento (media de los gradientes).
   - $v_t$ es el segundo momento (media de los cuadrados de los gradientes).
   - $\beta_1$ y $\beta_2$ son factores de decaimiento exponencial para los momentos estimados (típicamente 0.9 y 0.999).

3. **Corrección de sesgo**:
   $$\hat{m}_t = \frac{m_t}{1 - \beta_1^t}$$
   $$\hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$

4. **Actualizar los parámetros**:
   $$\theta_{t+1} = \theta_t - \frac{\rho \hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

   Donde $\epsilon$ es un pequeño número para evitar la división por cero (típicamente $10^{-8}$).

#### AdamW: Adam with Weight Decay

AdamW es una variante de Adam que desacopla el término de decaimiento de peso de la actualización de los parámetros. En Adam, el decaimiento de peso se aplica junto con la actualización de los parámetros, lo que puede llevar a una implementación incorrecta del decaimiento de peso. AdamW modifica la actualización de Adam para aplicar correctamente el decaimiento de peso:

$$\theta_{t+1} = \theta_t - \rho \left(\frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} + \lambda \theta_t\right)$$

Donde $\lambda$ es el coeficiente de decaimiento de peso.

Adam y AdamW son ampliamente utilizados en la práctica debido a su robustez y buen desempeño en una variedad de problemas de optimización en aprendizaje automático. Estos optimizadores son especialmente útiles cuando se trabaja con grandes conjuntos de datos y/o modelos de alta dimensionalidad, donde los métodos tradicionales como SGD puro pueden ser menos eficientes o converger lentamente.

En resumen, Adam y AdamW proporcionan mecanismos avanzados para ajustar los parámetros del modelo de manera más efectiva, adaptándose a las propiedades del gradiente y del problema específico, lo que a menudo resulta en una convergencia más rápida y estable en comparación con el uso de SGD con o sin momentum.


Referencia

https://www.ruder.io/optimizing-gradient-descent/